# NOTE
### CHeck this for basic outline of building a NN in pytorch - 
### https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

### https://machinelearningmastery.com/building-a-multiclass-classification-model-in-pytorch/

### https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html


# TODO

## 1. Add stop criteria for epoch. Training is working perfect
## 1. Integrate fusion framework with NN
## 1. Add full datasets. GPU trains super fast


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages `to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sarcasm/bert_embeddings_768.parquet
/kaggle/input/sarcasm/non_sarcasm_tweets.csv
/kaggle/input/sarcasm/sarcasm_tweets.csv
/kaggle/input/sarcasm/clean_raw_data.parquet
/kaggle/input/artkdata-private/clean_data_p2_w_bert.parquet
/kaggle/input/artkdata-private/ATRK dataset.csv
/kaggle/input/artkdata-private/clean_raw_data.parquet


# Import libraries

In [2]:
# Set random seed

SEED = 1234509876

# Importing basic libraries
from zipfile import ZipFile
import os, sys
import re
import gc
import time
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from string import punctuation
import pyarrow as pa
import pyarrow.parquet as pq

%matplotlib inline

# Import huggingface library
import transformers

# Import NLTK
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from wordcloud import WordCloud
# Import models

import lightgbm as lgb
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

# Model selection
from sklearn.model_selection import RandomizedSearchCV

# Others

from tqdm import tqdm_notebook #Loads progressbars for various loops

import warnings

warnings.filterwarnings('ignore')

#####################
#Useful pandas settings

pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 160)
pd.set_option('display.max_colwidth', 40)
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Import pytorch stuff

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader

# Check and assign a GPU instance
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import dataset

In [4]:
# IMPORT EXPORT CELL FOR PARQUET

##########################################
# IMPORT
raw_data = pq.read_table('/kaggle/input/artkdata-private/clean_data_p2_w_bert.parquet').to_pandas()

# ##########################################
# # EXPORT
# temp = pd.DataFrame(bert_word_embeddings)
# temp_col_names = temp.columns.values
# temp_col_names = ["bert_emb_"+str(col) for col in temp_col_names]


# temp.columns = temp_col_names

# temp.to_parquet(path = '/kaggle/working/bert_embeddings.parquet')

In [5]:
raw_data.head()

,text,labels,clean_text,emoji_list,clean_text_lem,clean_text_lem_stop,bert_emb_768
0,Youll notice on the new episode comi...,sarcasm,Youll notice on the new episode comi...,[],Youll notice on the new episode comi...,Youll notice new episode coming week...,"[-0.191491037607193, 0.0230156742036..."
1,@politicalHEDGE @BernieSanders Well ...,sarcasm,SOME_ENTITY SOME_ENTITY Well arent y...,[],SOME_ENTITY SOME_ENTITY Well arent y...,SOME_ENTITY SOME_ENTITY Well arent l...,"[0.21377572417259216, 0.089411765336..."
2,@ajodom60 @DeniseAshbaugh @nenanatio...,sarcasm,SOME_ENTITY SOME_ENTITY SOME_ENTITY ...,[],SOME_ENTITY SOME_ENTITY SOME_ENTITY ...,SOME_ENTITY SOME_ENTITY SOME_ENTITY ...,"[0.1151486411690712, 0.0453656576573..."
3,#livestreaming #Modeling Merlin for ...,sarcasm,"Merlin for the . and if you like , a...",[],"Merlin for the . and if you like , a...","Merlin . like , .","[0.109009750187397, -0.2241743505001..."
4,You spin me right round! Wholesale? ...,sarcasm,You spin me right round wholesale I ...,[],You spin me right round wholesale I ...,You spin right round wholesale I Hav...,"[0.7060186266899109, 0.0676050409674..."


# Temporary features

In [6]:
raw_data['text_len'] = raw_data['clean_text'].apply(lambda x: len(x.split(" ")))
raw_data.head()

,text,labels,clean_text,emoji_list,clean_text_lem,clean_text_lem_stop,bert_emb_768,text_len
0,Youll notice on the new episode comi...,sarcasm,Youll notice on the new episode comi...,[],Youll notice on the new episode comi...,Youll notice new episode coming week...,"[-0.191491037607193, 0.0230156742036...",35
1,@politicalHEDGE @BernieSanders Well ...,sarcasm,SOME_ENTITY SOME_ENTITY Well arent y...,[],SOME_ENTITY SOME_ENTITY Well arent y...,SOME_ENTITY SOME_ENTITY Well arent l...,"[0.21377572417259216, 0.089411765336...",21
2,@ajodom60 @DeniseAshbaugh @nenanatio...,sarcasm,SOME_ENTITY SOME_ENTITY SOME_ENTITY ...,[],SOME_ENTITY SOME_ENTITY SOME_ENTITY ...,SOME_ENTITY SOME_ENTITY SOME_ENTITY ...,"[0.1151486411690712, 0.0453656576573...",23
3,#livestreaming #Modeling Merlin for ...,sarcasm,"Merlin for the . and if you like , a...",[],"Merlin for the . and if you like , a...","Merlin . like , .","[0.109009750187397, -0.2241743505001...",11
4,You spin me right round! Wholesale? ...,sarcasm,You spin me right round wholesale I ...,[],You spin me right round wholesale I ...,You spin right round wholesale I Hav...,"[0.7060186266899109, 0.0676050409674...",31


# Remove entries that did not process properly

BERT embeddings did not process for all entries. Dropping invalid entries (n=90) to avoid issues.

In [7]:
raw_data.shape

(30000, 8)

In [8]:
raw_data = raw_data.loc[~raw_data['bert_emb_768'].isna(),:]

raw_data.shape

(29910, 8)

# Split dataset and assign the data loader

In [9]:
target_labels = raw_data.loc[:,'labels']

target_labels = (target_labels == "sarcasm").astype(int)
target_labels

0        1
1        1
2        1
3        1
4        1
        ..
29995    0
29996    0
29997    0
29998    0
29999    0
Name: labels, Length: 29910, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(raw_data, train_size=0.8, stratify=raw_data['labels'])

In [11]:
# Check if stratified samples are good

train_data.groupby('labels').count()['text'],test_data.groupby('labels').count()['text'] 

(labels
 non-sarcasm    12789
 sarcasm        11139
 Name: text, dtype: int64,
 labels
 non-sarcasm    3197
 sarcasm        2785
 Name: text, dtype: int64)

# Make dataloader to manage input

1. Extract required columns and assign them to a proper dataloader
1. Tweak dataloader if new features are added

Link - https://stackoverflow.com/questions/51444059/how-to-iterate-over-two-dataloaders-simultaneously-using-pytorch

In [12]:
# Quick function to auto assign required coluimns and make a tensor dataset
# class CustomTensorDataset(Dataset):
#     def __init__(self, dataset):
#         super(CustomTensorDataset, self).__init__()
        
#         self.dataset = dataset
#         self.target = dataset[0] # Targets will be stored first
#         self.base_embeddings = dataset[1] # Basic embeddings will be index 1
#         self.bert_emb_cols = dataset[2] # BERT embeddings will be index 2
#         self.add_feat_cols = dataset[3] # additional features will be index 3
    
#     def __len__(self):
#         return len(self.__len__)

#     def __getitem__(self, idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()

#         img_name = os.path.join(self.root_dir,
#                                 self.landmarks_frame.iloc[idx, 0])
#         image = io.imread(img_name)
#         landmarks = self.landmarks_frame.iloc[idx, 1:]
#         landmarks = np.array([landmarks])
#         landmarks = landmarks.astype('float').reshape(-1, 2)
#         sample = {'image': image, 'landmarks': landmarks}

#         if self.transform:
#             sample = self.transform(sample)

#         return sample
    
        
def usable_numpy_mat(df, base_emb_cols, bert_emb_cols, add_feat_cols, target):
    # Take a dataframe and given column haeaders and convert it to simple to use pytorch dataset
    # Will retain order of BERT EMBS >>> BASE EMBS/BASE EMB PREDS >>> ADD FEATUIRES
    # Returns 2 arrays, 1 with all features, 1 with targets
    
    # NOT NEEDED ANYMORE
    # lambda func for converting embeddings stored in a single column to numpy array
    # emb_column_to_numpy = lambda array: np.vstack(a[0] for a in array.values) 
    
    # One column will have numpy arrays with embeddings
    base_embs = np.vstack(df.loc[:,base_emb_cols].values)
    bert_embs = np.vstack(df.loc[:,bert_emb_cols].values)
    
    # One column per feature present
    add_feats = df.loc[:,add_feat_cols].to_numpy()
    
    targets = (df.loc[:,'labels'] == 'sarcasm').astype(int).to_numpy()
    
    # Merge all features into one numpy frame
    all_feats = np.column_stack((bert_embs, base_embs, add_feats))
    
    return all_feats, targets
    

In [13]:


train_feat_arr, train_tgt_arr = usable_numpy_mat(df = train_data,
                                    base_emb_cols = ('bert_emb_768'),
                                    bert_emb_cols = ('bert_emb_768'),
                                    add_feat_cols = ('text_len'), 
                                    target = ('labels'))


test_feat_arr, test_tgt_arr = usable_numpy_mat(df = test_data,
                                    base_emb_cols = ('bert_emb_768'),
                                    bert_emb_cols = ('bert_emb_768'),
                                    add_feat_cols = ('text_len'), 
                                    target = ('labels'))

###
#REF
# targets = torch.from_numpy((df.loc[:,'labels'] == 'sarcasm').astype(int).to_numpy()).to(device)
    
train_dataset_tensor = torch.utils.data.TensorDataset(torch.from_numpy(train_feat_arr),torch.from_numpy(train_tgt_arr))
test_dataset_tensor = torch.utils.data.TensorDataset(torch.from_numpy(test_feat_arr),torch.from_numpy(test_tgt_arr))

In [14]:
train_data_loader = torch.utils.data.DataLoader(dataset = train_dataset_tensor, batch_size = 128, shuffle= True)
test_data_loader = torch.utils.data.DataLoader(dataset = test_dataset_tensor, batch_size = 128, shuffle= True)

# Make the BNeural Network

In [15]:
# Base model will use basic word embeddings to supply a prediction.

def multi_framework_classifier(base_data_in, fused_data_in, target_labels, base_model_in, fused_model_in):
    # WIP
    # Base data is only expected to have word embeddings for the base model
    # The fused data will have features + embeddings
    # Labels will be common for both datasets, so ensure order does not change
    
    ##################################################
    # Copy of inputs to avoid replacing anything
    base_model = base_model_in
    fused_model = fused_model_in
    
    base_data = base_data_in
    fused_data = fused_data_in
    target_labels = target_labels_in
    
    # Fit each model on data provided
    base_pred = base_model.fit(base_data, target_labels)
    fused_pred = fused_model.fit(fused_data, target_labels)


In [16]:
class NeuralNetwork(nn.Module):
    def __init__(self, n_input_feats):
        # Defualt initiation fo Pytorch NN
        super(NeuralNetwork,self).__init__()
        
        # dEFINE DROPOUT FOR THE WHOLE NN
        self.drop = nn.Dropout(p = 0.3)
        self.input_layer_size = n_input_feats
        
        # Not sure what this does
        self.flatten = nn.Flatten()
        
        # Build sequential stack NN off below layers
        # 2 Hidden layer
        print(f"HEAR YE HEAR YE: Features = {self.input_layer_size} ")
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features = self.input_layer_size, out_features = 256),
            nn.ReLU(), # ReLU asctivation to replace sigmoid
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 2),
        )
        self.out = nn.Softmax(dim = 1) # SOFT MAX ON EACH ROW

    def forward(self, x):
        x = self.flatten(x) # Does not matter since we have word embeddings, but keeping it in
        logits = self.linear_relu_stack(x.float())
        probs = self.out(logits) #Convert to probablities
        
        return probs
    

# class Classifier_NN(nn.Module):
    
#     def __init__(self, n_classes):
#         # Not sure what this does exactlty.
#         # Best guess is we inherit the properties from torch and initiate it
#         super(Classifier_NN, self).__init__()
        
#         # Define dropout probablity for NN
#         self.drop = nn.Dropout(p = 0.3)
        
#         # Define output layer
#         # Hardcoded input size. Change LATER
#         # Output classes will be just 2
#         self.out = nn.Linear(in_features = 768, out_features= n_classes)
        
#         # We will use softmax for output
#         self.softmax = nn.Softmax(dim = 1)
    
#     def forward(self, ):
#         output = self.drop(embedding) # Apply dropout
#         output = self.out(output.float()) # Run input through network
        
#         return self.softmax(output) # Return softmax converted output
        

# Make a training function

In [17]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    # Make model into training mode
    model = model.train()
    
    # Save losses for each batch and number of correct predctions
    losses = []
    correct_preds = 0
    
    for d in data_loader:
        # iterate using a data loader
        # Save dataset portions into variables
        
        all_feats = d[0].to(device)      
        targets = d[1].to(device)
        
        #Generate predictions
        outputs = model(all_feats)
        
        # Generate predictions
        _, preds = torch.max(outputs, dim = 1)
        # Store losses
        loss = loss_fn(outputs, targets)
        
        # Storing current loop info for tracking
        correct_preds += torch.sum(preds == targets)
        losses.append(loss.item())
        
        # Backprop in model
        loss.backward()
        # Compensate for exploding gradients
        # Using the version of function that works inplace
        nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)
        # Step in optimizer and scheduler
        optimizer.step()
        scheduler.step()
        # Unsure what this does
        optimizer.zero_grad()
    
    # Return accuracy and mean of all losses
    return correct_preds.double()/n_examples, np.mean(losses)
        

# Make evaluation function for model

This is copy of training function without the steppers

In [18]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    
    model = model.eval() # Switch to eval mode so no gradient update
    
    losses = []
    correct_preds = 0
    with torch.no_grad():
        for d in data_loader:
            # iterate using a data loader
            # Save dataset portions into variables

            all_feats = d[0].to(device)      
            targets = d[1].to(device)

            #Generate predictions
            outputs = model(all_feats)

            # Generate predictions
            _, preds = torch.max(outputs, dim = 1)
            # Store losses
            loss = loss_fn(outputs, targets)

            # Storing current loop info for tracking
            correct_preds += torch.sum(preds == targets)
            losses.append(loss.item())


## Training

In [19]:
train_feat_arr.shape[0]

23928

In [20]:
EPOCHS = 200

n_train_samples = train_feat_arr.shape[0]
n_input_feats = train_feat_arr.shape[1]

nn_model = NeuralNetwork(n_input_feats) # Define a new neural network based on previous function
nn_model = nn_model.to(device) # Attach network to GPU

# Define optiizer for trainign NN
optimizer = transformers.AdamW(nn_model.parameters(), lr = 2e-5, correct_bias = False)

# Define total number of steps
total_train_steps = n_train_samples * EPOCHS # Number of epochs

# Define a scheduler
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer
    ,num_warmup_steps= 0
    ,num_training_steps= total_train_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

HEAR YE HEAR YE: Features = 1537 


In [21]:
from collections import defaultdict

old_acc, old_loss

for epoch in range(EPOCHS):
    
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)
    
    train_acc, train_loss = train_epoch(nn_model, train_data_loader, loss_fn, optimizer, device, scheduler,n_train_samples)
    
    print(f'Accuracy:{round(float(train_acc),5)} | Loss:{round(float(train_loss),5)}')

Epoch 1/2000
----------
Accuracy:0.77808 | Loss:0.54459
Epoch 2/2000
----------
Accuracy:0.82092 | Loss:0.48864
Epoch 3/2000
----------
Accuracy:0.82945 | Loss:0.48018
Epoch 4/2000
----------
Accuracy:0.83375 | Loss:0.47536
Epoch 5/2000
----------
Accuracy:0.83697 | Loss:0.47181
Epoch 6/2000
----------
Accuracy:0.84069 | Loss:0.46917
Epoch 7/2000
----------
Accuracy:0.84274 | Loss:0.46673
Epoch 8/2000
----------
Accuracy:0.84504 | Loss:0.46466
Epoch 9/2000
----------
Accuracy:0.84612 | Loss:0.46303
Epoch 10/2000
----------
Accuracy:0.84967 | Loss:0.46131
Epoch 11/2000
----------
Accuracy:0.85105 | Loss:0.45948
Epoch 12/2000
----------
Accuracy:0.85306 | Loss:0.45778
Epoch 13/2000
----------
Accuracy:0.85502 | Loss:0.4562
Epoch 14/2000
----------
Accuracy:0.85724 | Loss:0.45462
Epoch 15/2000
----------
Accuracy:0.85937 | Loss:0.4531
Epoch 16/2000
----------
Accuracy:0.86071 | Loss:0.45156
Epoch 17/2000
----------
Accuracy:0.86255 | Loss:0.4505
Epoch 18/2000
----------
Accuracy:0.86459 |

KeyboardInterrupt: 

In [ ]:
base_model.fit(base_data, target_labels)

preds = base_model.predict(base_data)
preds

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(target_labels, preds)

accuracy